### Demo: Learning bond sizes for random 1D Heisenberg Hamiltonians.

In [1]:
import h5py
# import numpy as np
import pandas as pd
fpath = "./mps_heis_data.h5"

import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader


import copy

In [2]:
# Read in Matt's data
f = h5py.File(fpath, 'r')

cmt='''
File mps_heis_data.h5 has one array of shape (10121, 50, 4)
- index 1 runs over the 10121 random instances of heisenberg spin-1/2 H's. Each H is of the form
H = \sum_{j, j+1} J_j S_j \dot S_{j+1} + \sum_j h_j S^z_j
	- the couplings J_j are random gaussian variables with mean -1 (antiferromagnetic) and variance 0.1
	- the fields h_j are random gaussian variables with mean 0 and variance 0.1
	- the chain is of length L = 50
- index 2 runs over the physical index {j} of the spin chain
- index 3 labels the stored data as follows:
	- k == 0: vector of the J_j's. This is of size 49 (open boundary conditions) so there is a zero prepended
		e.g. element (100, 29, 0) stores the value of J_{28} which is the coupling between the 28th and 29th qubit
	- k == 1: vector of the h_j's. This is of size 50
	- k == 2: vector of the *uncompressed* bond dimension of the ground state MPS |psi>
	- k == 3: vector of the *compressed* BD of an approximate ground state MPS |phi>

The compressed approximation |phi> satisfies |<phi | psi>| > 1 - 1e-6.

Notes on DMRG:
	- The DRMG algorithm that produced |psi> kept all singular values above 1e-12
	- Convergence was deemed achieved at an additive energy tolerance of 1e-4.
'''

In [4]:
data = torch.tensor(f['heis-bd-data'][:,:,:])
# np.array(data)
print(data)
Nsites = 50

tensor([[[-1.0882e+00, -3.7055e-01,  2.0000e+00,  2.0000e+00],
         [-9.3708e-01, -2.2749e-01,  4.0000e+00,  4.0000e+00],
         [-6.4150e-01, -5.5574e-01,  7.0000e+00,  4.0000e+00],
         ...,
         [-1.2262e+00,  2.9413e-01,  4.0000e+00,  4.0000e+00],
         [-9.9365e-01, -5.6239e-02,  2.0000e+00,  2.0000e+00],
         [ 0.0000e+00,  1.2912e-01,  0.0000e+00,  0.0000e+00]],

        [[-6.5802e-01, -1.3014e-01,  2.0000e+00,  2.0000e+00],
         [-7.7733e-01,  2.3377e-02,  4.0000e+00,  4.0000e+00],
         [-9.8828e-01, -4.4454e-01,  8.0000e+00,  7.0000e+00],
         ...,
         [-1.2862e+00,  1.6307e-01,  4.0000e+00,  4.0000e+00],
         [-1.0901e+00,  6.4691e-01,  2.0000e+00,  2.0000e+00],
         [ 0.0000e+00, -1.3289e-01,  0.0000e+00,  0.0000e+00]],

        [[-1.3416e+00,  2.8993e-02,  2.0000e+00,  2.0000e+00],
         [-8.7763e-01, -6.1247e-01,  4.0000e+00,  4.0000e+00],
         [-1.2726e+00,  6.8783e-02,  8.0000e+00,  8.0000e+00],
         ...,
         

In [6]:
# Get max bonds, for coarse-graining later
print( data.shape )
data[0,:,2]

max_dmrg_bond = torch.max(data[:,:,2])
print(f'max dmrg bond: {max_dmrg_bond}')

max_overlap_bond = torch.max(data[:,:,3])
print(f'max overlap bond: {max_overlap_bond}')


torch.Size([10121, 50, 4])
max dmrg bond: 226.0
max overlap bond: 97.0


In [36]:
# Coarse-grain the data

nslots = 30.

def coarsegrain_dmrg_bonddims(inp):
    return torch.round( (nslots/max_dmrg_bond)*inp )#.type(torch.int)

def uncoarsegrain_dmrg_bonddims(inp):
    return torch.round( (max_dmrg_bond/nslots)*inp )#.type(torch.int)

def coarsegrain_overlap_bonddims(inp):
    return torch.round( (nslots/max_overlap_bond)*inp )#.type(torch.int)

def uncoarsegrain_overlap_bonddims(inp):
    return torch.round( (max_overlap_bond/nslots)*inp )#.type(torch.int)

'''
# Quick tests:
print( coarsegrain_dmrg_bonddims(np.array([10,20,30])) )
print( uncoarsegrain_dmrg_bonddims(np.array([10,20,30])) )
print( coarsegrain_overlap_bonddims(np.array([10,20,30])) )
print( uncoarsegrain_overlap_bonddims(np.array([10,20,30])) )
'''

# Coarse-graining implemented here
coarsegr_data = data.type(torch.float32)
coarsegr_data[:,:,2] = coarsegrain_dmrg_bonddims(data[:,:,2])
coarsegr_data[:,:,3] = coarsegrain_overlap_bonddims(data[:,:,2])


# Split into training and testing data (do 85%-15%)
frac_train = 0.85
size_train = int(frac_train*coarsegr_data.shape[0])
print(size_train)
training_data = coarsegr_data[:size_train,:,:]
testing_data  = coarsegr_data[size_train:,:,:]

print(training_data.shape)
print(testing_data.shape)

8602
torch.Size([8602, 50, 4])
torch.Size([1519, 50, 4])


In [37]:


# class CustomImageDataset(Dataset):
#     def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
#         self.img_labels = pd.read_csv(annotations_file)
#         self.img_dir = img_dir
#         self.transform = transform
#         self.target_transform = target_transform

#     def __len__(self):
#         return len(self.img_labels)

#     def __getitem__(self, idx):
#         img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#         image = read_image(img_path)
#         label = self.img_labels.iloc[idx, 1]
#         if self.transform:
#             image = self.transform(image)
#         if self.target_transform:
#             label = self.target_transform(label)
#         return image, label

    

In [38]:
# Need to define DataSet class
# Must *use* a DataLoader, but don't need new class

class HeisDMRGbondsDataset(Dataset):
    
    def __init__(self,inp_data, transform=None, target_transform=None):
        self.raw_data = inp_data
        
    def __len__(self):
        return self.raw_data.shape[0]
    
    def __getitem__(self,idx):
        
        did = 20
        
        hamdata = self.raw_data[idx, :, 0:2]
#         hamdata = torch.expand_dims(self.raw_data[idx, :, 0:2],axis=0)
#         hamdata = torch.array([self.raw_data[idx, :, 0:2]])

        dvals   = self.raw_data[idx, :, 2] #[idx, :, 2:3]
#         dval   = self.raw_data[idx, did, 2] #[idx, :, 2:3]
        
        return hamdata, dvals#.astype(int)
#         return hamdata, int(dval)
        
        
# class HeisOverlapBondsDataset(Dataset):
        
        

In [39]:
# 'reg' means 'regular bonds'
# Note 'as type' to go to 32-bit float
reg_training_data = HeisDMRGbondsDataset(training_data)
reg_testing_data  = HeisDMRGbondsDataset(testing_data)

In [40]:
# Make sure output looks correct
print( reg_training_data[0] )
print( reg_testing_data[0] )

(tensor([[-1.0882, -0.3706],
        [-0.9371, -0.2275],
        [-0.6415, -0.5557],
        [-0.6356,  0.0032],
        [-1.2323,  0.1899],
        [-1.1554,  0.2513],
        [-0.9041, -0.4830],
        [-0.8500,  0.7034],
        [-1.2217,  0.4572],
        [-0.7484, -0.6532],
        [-1.1959,  0.0681],
        [-1.2091,  0.1533],
        [-0.9521,  0.1105],
        [-1.7078,  0.3483],
        [-0.4551, -0.0738],
        [-1.3972,  0.0123],
        [-1.1474,  0.0273],
        [-0.8863, -0.7602],
        [-1.2586, -0.8437],
        [-1.5607,  0.2770],
        [-1.4163, -0.2554],
        [-1.2745, -0.0767],
        [-0.9692, -0.1978],
        [-0.8101, -0.0828],
        [-0.6671,  0.1417],
        [-0.8453, -0.3644],
        [-0.9117,  0.4260],
        [-0.9629, -0.2203],
        [-0.8452, -0.2369],
        [-0.8597, -0.1995],
        [-0.9924, -0.5638],
        [-0.9245, -0.0166],
        [-0.6561,  0.0212],
        [-1.1099, -0.0297],
        [-0.6324,  0.4260],
        [-1.5027,  

In [41]:
# Create DataLoaders
batch_size = 64

# Create data loaders.
reg_train_dataloader = DataLoader(reg_training_data,batch_size=batch_size)
reg_test_dataloader = DataLoader(reg_testing_data,batch_size=batch_size)

for X, y in reg_test_dataloader:
    print(f"Shape of X: {X.shape} {X.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


print(nslots)

Shape of X: torch.Size([64, 50, 2]) torch.float32
Shape of y: torch.Size([64, 50]) torch.float32
30.0


In [42]:
# Create the model

print("try *not* doing nslots for now... try to learn d values directly...")

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(Nsites*2, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
#             nn.Linear(512, int(nslots) )
            nn.Linear(512, Nsites )
            
        )

    def forward(self, x):
#         print(f"x: {x}")
        x = self.flatten(x)
#         x = nn.Flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

try *not* doing nslots for now... try to learn d values directly...
Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=50, bias=True)
  )
)


In [43]:
def custom_loss(y_pred,y_val):
    return torch.sum(torch.absolute(y_pred - y_val))
loss_fn = custom_loss


# loss_fn = nn.CrossEntropyLoss() # <-- not appropriate for us
# loss_fn = nn.L1Loss
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # lr = learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # lr = learning rate

In [44]:
# Train function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
#             print('pred: ',X)
#             print('y: ',y)
            

In [45]:
# Test function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [46]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(reg_train_dataloader, model, loss_fn, optimizer)
#     test(reg_test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 16190.856445  [    0/ 8602]
loss: 7045.499512  [ 1280/ 8602]
loss: 6800.855469  [ 2560/ 8602]
loss: 5757.700195  [ 3840/ 8602]
loss: 6297.056152  [ 5120/ 8602]
loss: 6406.628418  [ 6400/ 8602]
loss: 6055.835938  [ 7680/ 8602]
Epoch 2
-------------------------------
loss: 7093.830566  [    0/ 8602]
loss: 6645.979492  [ 1280/ 8602]
loss: 6623.170898  [ 2560/ 8602]
loss: 5618.173828  [ 3840/ 8602]
loss: 6205.225586  [ 5120/ 8602]
loss: 6314.202148  [ 6400/ 8602]
loss: 6004.837891  [ 7680/ 8602]
Epoch 3
-------------------------------
loss: 7043.664062  [    0/ 8602]
loss: 6675.809570  [ 1280/ 8602]
loss: 6637.380859  [ 2560/ 8602]
loss: 5620.492676  [ 3840/ 8602]
loss: 6246.716797  [ 5120/ 8602]
loss: 6367.268555  [ 6400/ 8602]
loss: 6017.033203  [ 7680/ 8602]
Epoch 4
-------------------------------
loss: 7091.979980  [    0/ 8602]
loss: 6654.039551  [ 1280/ 8602]
loss: 6593.653809  [ 2560/ 8602]
loss: 5621.009766  [ 3840/ 8602]
loss: 6193.525

In [ ]:
# torch.save(model.state_dict(), "model.pth")
# print("Saved PyTorch Model State to model.pth")

In [ ]:
# model = NeuralNetwork()
# model.load_state_dict(torch.load("model.pth"))

In [49]:
model.eval()
x, y = reg_testing_data[0][0], reg_testing_data[0][1] 
# print(x)
# print(y) # true value
with torch.no_grad():
#     pred = model(x)
    pred = model(torch.flatten(x))
    print(pred) 
#     predicted, actual = pred[0].argmax(0), y # It's like one-hot. Position with *highest* value gives answer
    predicted, actual = pred[0], y
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)